# Курс Введение в обработку естественного языка / Практическое задание урока 5. Part-of-Speech разметка, NER, извлечение отношений

## Задание 1

**Написать теггер на данных с руским языком**
1. проверить UnigramTagger, BigramTagger, TrigramTagger и их комбмнации  
2. написать свой теггер как на занятии, но улучшить попробовать разные векторайзеры, добавить знание не только букв и слов но и совместно объединить эти признаки  
3. вместо векторайзеров взять эмбединги попробовать (word2vec и fasttext по желанию дополнительно можно взять tf.keras.layers.Embedding)  
4. взять не только эмбединги каждого слова, но и взять соседей, т.е. информацию о соседях количество соседей выбрать самим (узнать наилучшее количество соседей)    
5. сравнить все реализованные методы сделать выводы

### загрузка данных

In [1]:
!pip install pyconll

In [2]:
import pyconll

In [3]:
!mkdir datasets

In [4]:
!wget -O ./datasets/ru_syntagrus-ud-train.conllu https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train.conllu
!wget -O ./datasets/ru_syntagrus-ud-dev.conllu https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-dev.conllu

--2021-07-07 18:04:19--  https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train.conllu
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 81039282 (77M) [text/plain]
Saving to: ‘./datasets/ru_syntagrus-ud-train.conllu’

./datasets/ru_synta 100%[===================>]  77.28M   106MB/s    in 0.7s    

2021-07-07 18:04:21 (106 MB/s) - ‘./datasets/ru_syntagrus-ud-train.conllu’ saved [81039282/81039282]

--2021-07-07 18:04:21--  https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-dev.conllu
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercont

In [5]:
full_train = pyconll.load_from_file('datasets/ru_syntagrus-ud-train.conllu')
full_test = pyconll.load_from_file('datasets/ru_syntagrus-ud-dev.conllu')

In [6]:
for sent in full_train[:2]:
    for token in sent:
        print(token.form, token.upos)
    print()

Анкета NOUN
. PUNCT

Начальник NOUN
областного ADJ
управления NOUN
связи NOUN
Семен PROPN
Еремеевич PROPN
был AUX
человек NOUN
простой ADJ
, PUNCT
приходил VERB
на ADP
работу NOUN
всегда ADV
вовремя ADV
, PUNCT
здоровался VERB
с ADP
секретаршей NOUN
за ADP
руку NOUN
и CCONJ
иногда ADV
даже PART
писал VERB
в ADP
стенгазету NOUN
заметки NOUN
под ADP
псевдонимом NOUN
" PUNCT
Муха NOUN
" PUNCT
. PUNCT



## Решение

In [7]:
import nltk

from nltk.corpus import brown
from nltk.tag import DefaultTagger
from nltk.tag import UnigramTagger
from nltk.tag import BigramTagger, TrigramTagger
from nltk.tag import RegexpTagger

In [8]:
def conver_to_test_data(full):
    res = []
    for sent in full:
        sub_res = []
        for token in sent:
            sub_res.append((token.form, token.upos))
        res.append(sub_res)
    return res

In [9]:
def conver_to_test_sent(full):
    res = []
    for sent in full:
        sub_res = []
        for token in sent:
            sub_res.append(token.form)
        res.append(sub_res)
    return res

In [10]:
train_data = conver_to_test_data(full_train)

In [11]:
test_data = conver_to_test_data(full_test)
print(test_data[0:10])

[[('Алгоритм', 'NOUN'), (',', 'PUNCT'), ('от', 'ADP'), ('имени', 'NOUN'), ('учёного', 'NOUN'), ('аль', 'PART'), ('-', 'PUNCT'), ('Хорезми', 'PROPN'), (',', 'PUNCT'), ('-', 'PUNCT'), ('точный', 'ADJ'), ('набор', 'NOUN'), ('инструкций', 'NOUN'), (',', 'PUNCT'), ('описывающих', 'VERB'), ('порядок', 'NOUN'), ('действий', 'NOUN'), ('исполнителя', 'NOUN'), ('для', 'ADP'), ('достижения', 'NOUN'), ('результата', 'NOUN'), ('решения', 'NOUN'), ('задачи', 'NOUN'), ('за', 'ADP'), ('конечное', 'ADJ'), ('время', 'NOUN'), ('.', 'PUNCT')], [('В', 'ADP'), ('старой', 'ADJ'), ('трактовке', 'NOUN'), ('вместо', 'ADP'), ('слова', 'NOUN'), ('"', 'PUNCT'), ('порядок', 'NOUN'), ('"', 'PUNCT'), ('использовалось', 'VERB'), ('слово', 'NOUN'), ('"', 'PUNCT'), ('последовательность', 'NOUN'), ('"', 'PUNCT'), (',', 'PUNCT'), ('но', 'CCONJ'), ('по', 'ADP'), ('мере', 'NOUN'), ('развития', 'NOUN'), ('параллельности', 'NOUN'), ('в', 'ADP'), ('работе', 'NOUN'), ('компьютеров', 'NOUN'), ('слово', 'NOUN'), ('"', 'PUNCT'), (

In [12]:
test_sent = conver_to_test_sent(full_test)[0]
print(test_sent)

['Алгоритм', ',', 'от', 'имени', 'учёного', 'аль', '-', 'Хорезми', ',', '-', 'точный', 'набор', 'инструкций', ',', 'описывающих', 'порядок', 'действий', 'исполнителя', 'для', 'достижения', 'результата', 'решения', 'задачи', 'за', 'конечное', 'время', '.']


### 1.Проверить UnigramTagger, BigramTagger, TrigramTagger и их комбинации

In [13]:
unigram_tagger = UnigramTagger(train_data)
display(unigram_tagger.tag(test_sent), unigram_tagger.evaluate(test_data))

[('Алгоритм', None),
 (',', 'PUNCT'),
 ('от', 'ADP'),
 ('имени', 'NOUN'),
 ('учёного', 'NOUN'),
 ('аль', 'PART'),
 ('-', 'PUNCT'),
 ('Хорезми', None),
 (',', 'PUNCT'),
 ('-', 'PUNCT'),
 ('точный', 'ADJ'),
 ('набор', 'NOUN'),
 ('инструкций', 'NOUN'),
 (',', 'PUNCT'),
 ('описывающих', 'VERB'),
 ('порядок', 'NOUN'),
 ('действий', 'NOUN'),
 ('исполнителя', 'NOUN'),
 ('для', 'ADP'),
 ('достижения', 'NOUN'),
 ('результата', 'NOUN'),
 ('решения', 'NOUN'),
 ('задачи', 'NOUN'),
 ('за', 'ADP'),
 ('конечное', None),
 ('время', 'NOUN'),
 ('.', 'PUNCT')]

0.8772537323492737

In [14]:
bigram_tagger = BigramTagger(train_data, backoff=unigram_tagger)
display(bigram_tagger.tag(test_sent), bigram_tagger.evaluate(test_data))

[('Алгоритм', None),
 (',', 'PUNCT'),
 ('от', 'ADP'),
 ('имени', 'NOUN'),
 ('учёного', 'NOUN'),
 ('аль', 'PART'),
 ('-', 'PUNCT'),
 ('Хорезми', None),
 (',', 'PUNCT'),
 ('-', 'PUNCT'),
 ('точный', 'ADJ'),
 ('набор', 'NOUN'),
 ('инструкций', 'NOUN'),
 (',', 'PUNCT'),
 ('описывающих', 'VERB'),
 ('порядок', 'NOUN'),
 ('действий', 'NOUN'),
 ('исполнителя', 'NOUN'),
 ('для', 'ADP'),
 ('достижения', 'NOUN'),
 ('результата', 'NOUN'),
 ('решения', 'NOUN'),
 ('задачи', 'NOUN'),
 ('за', 'ADP'),
 ('конечное', None),
 ('время', 'NOUN'),
 ('.', 'PUNCT')]

0.8829828463586425

In [15]:
trigram_tagger = TrigramTagger(train_data, backoff=bigram_tagger)
display(trigram_tagger.tag(test_sent), trigram_tagger.evaluate(test_data))

[('Алгоритм', None),
 (',', 'PUNCT'),
 ('от', 'ADP'),
 ('имени', 'NOUN'),
 ('учёного', 'NOUN'),
 ('аль', 'PART'),
 ('-', 'PUNCT'),
 ('Хорезми', None),
 (',', 'PUNCT'),
 ('-', 'PUNCT'),
 ('точный', 'ADJ'),
 ('набор', 'NOUN'),
 ('инструкций', 'NOUN'),
 (',', 'PUNCT'),
 ('описывающих', 'VERB'),
 ('порядок', 'NOUN'),
 ('действий', 'NOUN'),
 ('исполнителя', 'NOUN'),
 ('для', 'ADP'),
 ('достижения', 'NOUN'),
 ('результата', 'NOUN'),
 ('решения', 'NOUN'),
 ('задачи', 'NOUN'),
 ('за', 'ADP'),
 ('конечное', None),
 ('время', 'NOUN'),
 ('.', 'PUNCT')]

0.882081353418933

In [16]:
# Комбинация тэггеров
def backoff_tagger(train_sents, tagger_classes, backoff=None):
    for cls in tagger_classes:
        backoff = cls(train_sents, backoff=backoff)
    return backoff


backoff = DefaultTagger('NN') 
tag = backoff_tagger(train_data,  
                     [UnigramTagger, BigramTagger, TrigramTagger],  
                     backoff = backoff) 
  
tag.evaluate(test_data)

0.8814747413473528

### 2.Написать свой теггер как на занятии, но улучшить попробовать разные векторайзеры, добавить знание не только букв и слов но и совместно объединить эти признаки

In [17]:
from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer, TfidfVectorizer

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

In [18]:
fdata_train = []
for sent in full_train[:]:
    fdata_train.append([(token.form.lower(), token.upos) for token in sent])
    
fdata_test = []
for sent in full_test[:]:
    fdata_test.append([(token.form.lower(), token.upos) for token in sent])
    
fdata_sent_train = []
for sent in full_train[:]:
    fdata_sent_train.append([token.form.lower() for token in sent])
    
fdata_sent_test = []
for sent in full_test[:]:
    fdata_sent_test.append([token.form.lower() for token in sent])
    
    
MAX_SENT_LEN = max(len(sent) for sent in full_train)
MAX_ORIG_TOKEN_LEN = max(len(token.form) for sent in full_train for token in sent)
print('Наибольшая длина предложения', MAX_SENT_LEN)
print('Наибольшая длина токена', MAX_ORIG_TOKEN_LEN)

Наибольшая длина предложения 205
Наибольшая длина токена 47


In [19]:
train_tok = []
train_label = []
for sent in fdata_train[:]:
    for tok in sent:
        train_tok.append(tok[0])
        train_label.append('NO_TAG' if tok[1] is None else tok[1])
        
test_tok = []
test_label = []
for sent in fdata_test[:]:
    for tok in sent:
        test_tok.append(tok[0])
        test_label.append('NO_TAG' if tok[1] is None else tok[1])

le = LabelEncoder()
train_enc_labels = le.fit_transform(train_label) 
test_enc_labels = le.transform(test_label)
le.classes_

array(['ADJ', 'ADP', 'ADV', 'AUX', 'CCONJ', 'DET', 'INTJ', 'NOUN',
       'NO_TAG', 'NUM', 'PART', 'PRON', 'PROPN', 'PUNCT', 'SCONJ', 'SYM',
       'VERB', 'X'], dtype='<U6')

In [20]:
# берем разные векторайзеры, анализируем по буквам
for vectorizer in [CountVectorizer, HashingVectorizer, TfidfVectorizer]:

    scaler = StandardScaler(with_mean=False)
    coder = vectorizer(ngram_range=(1, 5), analyzer='char')
    

    X_train = coder.fit_transform(train_tok)
    X_test = coder.transform(test_tok)
    
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.fit_transform(X_test)    
    
    
    print(X_train.shape)
    lr = LogisticRegression(random_state=0, max_iter = 100, n_jobs=7)
    lr.fit(X_train, train_enc_labels)

    pred = lr.predict(X_test)

    print(vectorizer, accuracy_score(test_enc_labels, pred))

(871526, 149809)
<class 'sklearn.feature_extraction.text.CountVectorizer'> 0.943644053516665
(871526, 1048576)
<class 'sklearn.feature_extraction.text.HashingVectorizer'> 0.9471826239342163
(871526, 149809)
<class 'sklearn.feature_extraction.text.TfidfVectorizer'> 0.9487749806221144


In [21]:
# берем разные векторайзеры, анализируем по словам
for vectorizer in [CountVectorizer, HashingVectorizer, TfidfVectorizer]:

    scaler = StandardScaler(with_mean=False)
    coder = vectorizer(ngram_range=(1, 5), analyzer='word')
    

    X_train = coder.fit_transform(train_tok)
    X_test = coder.transform(test_tok)
    
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.fit_transform(X_test)    
    
    
    print(X_train.shape)
    lr = LogisticRegression(random_state=0, max_iter = 100, n_jobs=7)
    lr.fit(X_train, train_enc_labels)

    pred = lr.predict(X_test)

    print(vectorizer, accuracy_score(test_enc_labels, pred))

(871526, 99485)
<class 'sklearn.feature_extraction.text.CountVectorizer'> 0.7531847133757962
(871526, 1048576)
<class 'sklearn.feature_extraction.text.HashingVectorizer'> 0.7722255922892866
(871526, 99485)
<class 'sklearn.feature_extraction.text.TfidfVectorizer'> 0.7532605398847437


In [22]:
# В случае совмещения двух признаков (буквы, слова) код успешно отработал только при первом запуске
# При последующих запусках выдавал ошибку. 
# Для ликвидации ошибки делалось уменьшение n_jobs. 
# После запуска в другой день и установки  n_jobs=4 ошибка не выдалась.
import numpy as np
from scipy.sparse import hstack

# берем векторайзер TfidfVectorizer по буквам и HashingVectorizer по словам
scaler = StandardScaler(with_mean=False)
coder_1 = TfidfVectorizer(ngram_range=(1, 5), analyzer='char')
coder_2 = HashingVectorizer(ngram_range=(1, 5), analyzer='word')

X_train_1 = coder_1.fit_transform(train_tok)
X_test_1 = coder_1.transform(test_tok)

X_train_2 = coder_2.fit_transform(train_tok)
X_test_2 = coder_2.transform(test_tok)


X_train = hstack((X_train_1,X_train_2))
X_test = hstack((X_test_1,X_test_2))

X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)    


print(X_train.shape)
lr = LogisticRegression(random_state=0, max_iter = 100, n_jobs=4)
lr.fit(X_train, train_enc_labels)

pred = lr.predict(X_test)

print('TfidfVectorizer_char + HashingVectorizer_word :', accuracy_score(test_enc_labels, pred))

(871526, 1198385)
TfidfVectorizer_char + HashingVectorizer_word : 0.9441327132409935


### 3.Вместо векторайзеров взять эмбединги попробовать (word2vec и fasttext по желанию дополнительно можно взять tf.keras.layers.Embedding)

In [23]:
import gensim

#### word2vec

In [24]:
train_word2vec = fdata_sent_train
train_word2vec.extend(fdata_sent_test)

In [25]:
w2v_model = gensim.models.Word2Vec(min_count=1, negative=20, sg=1, window=2)
w2v_model.build_vocab(train_word2vec)
w2v_model.train(train_word2vec, epochs=5, total_examples=w2v_model.corpus_count)

(3850208, 4951090)

In [26]:
X_train_w2v = [w2v_model.wv[word] for word in train_tok]
X_test_w2v = [w2v_model.wv[word] for word in test_tok]

In [27]:
lr = LogisticRegression(random_state=0, max_iter=35) # max_iter изменено с 25 на 35
lr.fit(X_train_w2v, train_enc_labels)
pred = lr.predict(X_test_w2v)
accuracy_score(test_enc_labels, pred)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.7469837899774205

#### fasttext

In [28]:
ft_model = gensim.models.FastText(train_word2vec, min_count=1, negative=20, sg=1, window=5)

In [29]:
X_train_ft = [ft_model.wv[word] for word in train_tok]
X_test_ft = [ft_model.wv[word] for word in test_tok]

In [30]:
lr = LogisticRegression(random_state=0, max_iter=35) # max_iter изменено с 25 на 35
lr.fit(X_train_ft, train_enc_labels)
pred = lr.predict(X_test_ft)
accuracy_score(test_enc_labels, pred)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.8727631179860479

#### 4.взять не только эмбединги каждого слова, но и взять соседей, т.е. информацию о соседях количество соседей выбрать самим (узнать наилучшее количество соседей)

In [31]:
import warnings
warnings.filterwarnings("ignore")

In [32]:
lr = LogisticRegression(random_state=0, max_iter=25)

In [33]:
windows = [1,2,3,4,5,6,7,8,9]
for i in windows:
    w2v_model = gensim.models.Word2Vec(min_count=1, negative=20, sg=1, window=i)
    w2v_model.build_vocab(train_word2vec)
    w2v_model.train(train_word2vec, epochs=5, total_examples=w2v_model.corpus_count)
    X_train_w2v = [w2v_model.wv[word] for word in train_tok]
    X_test_w2v = [w2v_model.wv[word] for word in test_tok]
    lr.fit(X_train_w2v, train_enc_labels)
    pred = lr.predict(X_test_w2v)
    print('word2vec, window=', i, accuracy_score(test_enc_labels, pred))

word2vec, window= 1 0.7483992181444411
word2vec, window= 2 0.7394432649209719
word2vec, window= 3 0.7370505173052944
word2vec, window= 4 0.7279766117345735
word2vec, window= 5 0.7279681865669126
word2vec, window= 6 0.7235449735449735
word2vec, window= 7 0.7155242139318573
word2vec, window= 8 0.7166868870690527
word2vec, window= 9 0.7158696458059516


In [ ]:
windows = [1,2,3,4,5,6,7,8,9]
for i in windows:
    ft_model = gensim.models.FastText(train_word2vec, min_count=1, negative=20, sg=1, window=i)
    X_train_ft = [ft_model.wv[word] for word in train_tok]
    X_test_ft = [ft_model.wv[word] for word in test_tok]
    lr.fit(X_train_ft, train_enc_labels)
    pred = lr.predict(X_test_ft)
    print('fasttext, window=', i, accuracy_score(test_enc_labels, pred))

fasttext, window= 1 0.8582886799447309
fasttext, window= 2 0.864000943618778
fasttext, window= 3 0.8595440299261955
fasttext, window= 4 0.8590553702018671
fasttext, window= 5 0.8587015131601119
fasttext, window= 6 0.851380042462845
fasttext, window= 7 0.8503100461699188
fasttext, window= 8 0.8453476224176861
fasttext, window= 9 0.8446651838371584


#### 5.сравнить все реализованные методы сделать выводы

* 0.88147 - скор по комбинации из UnigramTagger BigramTagger TrigramTagger при методе nltk.tag
* 0.94877 - скор LogisticRegression в сочетании с TfidfVectorizer по буквам при методе Vectorizer
* 0.86400 - скор LogisticRegression в сочетании с  FastText по window= 2 при методе эмбедингов 

**Вывод:**
* для рассматриваемого корпуса наилучшим является исследование по буквам, а не по словам 





# Задание 2

много дополнительных датасетов на русском языке

https://natasha.github.io/corus/  
https://github.com/natasha/corus

мы будем использовать данные http://www.labinform.ru/pub/named_entities/

**Проверить насколько хорошо работает NER**

1. взять нер из nltk
2. проверить deeppavlov
3. написать свой нер попробовать разные подходы (с доп информацией без) так же с учётом соседей и без них
4. сделать выводы по вашим экспериментам какой из подходов успешнее справляется

при обучении своего нера не забудьте разделить выборку

In [34]:
!pip install corus

     |████████████████████████████████| 92kB 807kB/s 


In [35]:
import corus

In [36]:
!wget http://www.labinform.ru/pub/named_entities/collection5.zip

--2021-07-07 19:35:00--  http://www.labinform.ru/pub/named_entities/collection5.zip
Resolving www.labinform.ru (www.labinform.ru)... 80.240.100.4
Connecting to www.labinform.ru (www.labinform.ru)|80.240.100.4|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1899530 (1.8M) [application/zip]
Saving to: ‘collection5.zip’

collection5.zip     100%[===================>]   1.81M   411KB/s    in 4.5s    

2021-07-07 19:35:05 (411 KB/s) - ‘collection5.zip’ saved [1899530/1899530]



In [37]:
!unzip collection5.zip

Archive:  collection5.zip
   creating: Collection5/
  inflating: Collection5/001.ann     
  inflating: Collection5/001.txt     
  inflating: Collection5/002.ann     
  inflating: Collection5/002.txt     
  inflating: Collection5/003.ann     
  inflating: Collection5/003.txt     
  inflating: Collection5/004.ann     
  inflating: Collection5/004.txt     
  inflating: Collection5/005.ann     
  inflating: Collection5/005.txt     
  inflating: Collection5/006.ann     
  inflating: Collection5/006.txt     
  inflating: Collection5/007.ann     
  inflating: Collection5/007.txt     
  inflating: Collection5/008.ann     
  inflating: Collection5/008.txt     
  inflating: Collection5/009.ann     
  inflating: Collection5/009.txt     
  inflating: Collection5/010.ann     
  inflating: Collection5/010.txt     
  inflating: Collection5/011.ann     
  inflating: Collection5/011.txt     
  inflating: Collection5/012.ann     
  inflating: Collection5/012.txt     
  inflating: Collection5/013.ann    

In [38]:
!rm collection5.zip

In [39]:
!ls

Collection5  datasets  sample_data


In [40]:
from corus import load_ne5

dir = 'Collection5/'
records = load_ne5(dir)
rec = next(records)


In [41]:
rec.spans

[Ne5Span(
     index='T1',
     type='ORG',
     start=6,
     stop=8,
     text='СК'
 ), Ne5Span(
     index='T2',
     type='ORG',
     start=40,
     stop=56,
     text='Балтийском флоте'
 ), Ne5Span(
     index='T3',
     type='ORG',
     start=60,
     stop=80,
     text='Следственный комитет'
 ), Ne5Span(
     index='T4',
     type='ORG',
     start=163,
     stop=206,
     text='ОАО «Управление торговли Балтийского флота»'
 ), Ne5Span(
     index='T5',
     type='PER',
     start=207,
     stop=227,
     text='Максима Малаховского'
 ), Ne5Span(
     index='T6',
     type='PER',
     start=362,
     stop=375,
     text='М.Малаховский'
 ), Ne5Span(
     index='T7',
     type='ORG',
     start=505,
     stop=549,
     text='ООО «Калининградский межотраслевой комбинат»'
 ), Ne5Span(
     index='T8',
     type='ORG',
     start=551,
     stop=590,
     text='ООО «Балтийский межотраслевой комбинат»'
 ), Ne5Span(
     index='T9',
     type='ORG',
     start=593,
     stop=633,
     tex

In [42]:
!pip install razdel

In [43]:
from razdel import tokenize

In [44]:
words_docs = []
for ix, rec in enumerate(records):
    words = []
    for token in tokenize(rec.text):
        is_person = False
        for person in rec.spans:
            if (token.start >= person.start) and (token.stop <= person.stop):
                is_person = True
                break
        words.append([token.text, 'PERSON' if is_person else 'NO_PERSON'])
    words_docs.extend(words)

In [45]:
import pandas as pd

In [46]:
df_words = pd.DataFrame(words_docs, columns=['word', 'tag'])

In [47]:
df_words['tag'].value_counts()

NO_PERSON    218995
PERSON        46179
Name: tag, dtype: int64

In [48]:
df_words.head(20)

,word,tag
0,Шанцев,PERSON
1,скорректировал,NO_PERSON
2,структуру,NO_PERSON
3,нижегородского,NO_PERSON
4,правительства,NO_PERSON
5,и,NO_PERSON
6,назначил,NO_PERSON
7,себе,NO_PERSON
8,новых,NO_PERSON
9,заместителей,NO_PERSON


#### 1.взять нер из nltk

In [49]:
import nltk
from nltk import word_tokenize, pos_tag, ne_chunk

In [50]:
nltk.download('maxent_ne_chunker')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('words')

[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


True

In [51]:
nltk.pos_tag(nltk.word_tokenize(rec.text))

[('Вице-мэром', 'JJ'),
 ('Новосибирска', 'NNP'),
 ('по', 'NNP'),
 ('социальным', 'NNP'),
 ('вопросам', 'NNP'),
 ('стал', 'NNP'),
 ('учитель', 'VBD'),
 ('10:14', 'CD'),
 ('31.01.2013', 'CD'),
 ('Мэр', 'NN'),
 ('Новосибирска', 'NNP'),
 ('Владимир', 'NNP'),
 ('Городецкий', 'NNP'),
 ('назначил', 'NNP'),
 ('своим', 'NNP'),
 ('заместителем', 'NNP'),
 ('по', 'NNP'),
 ('социальным', 'NNP'),
 ('вопросам', 'NNP'),
 ('бывшего', 'NNP'),
 ('замначальника', 'NNP'),
 ('городского', 'NNP'),
 ('управления', 'NNP'),
 ('образования', 'NNP'),
 ('Сергея', 'NNP'),
 ('Нелюбова', 'NNP'),
 (';', ':'),
 ('главным', 'NNP'),
 ('финансистом', 'NNP'),
 ('города', 'NNP'),
 ('стал', 'NNP'),
 ('бывший', 'NNP'),
 ('первый', 'NNP'),
 ('замглавы', 'NNP'),
 ('Бердска', 'NNP'),
 ('Владимир', 'NNP'),
 ('Штоп', 'NNP'),
 ('.', '.')]

In [52]:
{(' '.join(c[0] for c in chunk), chunk.label() ) for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(rec.text))) if hasattr(chunk, 'label') }

{('Бердска Владимир Штоп', 'PERSON'),
 ('Новосибирска Владимир Городецкий', 'PERSON'),
 ('Сергея Нелюбова', 'PERSON')}

#### 2.проверить deeppavlov

In [53]:
# установка deeppavlov

!pip uninstall -y tensorflow tensorflow-gpu
!pip install numpy scipy librosa unidecode inflect librosa transformers
!pip install deeppavlov

Uninstalling tensorflow-2.5.0:
  Successfully uninstalled tensorflow-2.5.0
     |████████████████████████████████| 245kB 5.1MB/s 
     |████████████████████████████████| 2.5MB 19.8MB/s 
     |████████████████████████████████| 901kB 32.8MB/s 
     |████████████████████████████████| 3.3MB 36.8MB/s 
     |████████████████████████████████| 911kB 5.2MB/s 
     |████████████████████████████████| 61kB 6.7MB/s 
     |████████████████████████████████| 7.3MB 15.8MB/s 
     |████████████████████████████████| 3.8MB 36.6MB/s 
     |████████████████████████████████| 512kB 31.3MB/s 
     |████████████████████████████████| 860kB 19.0MB/s 
     |████████████████████████████████| 2.9MB 31.1MB/s 
     |████████████████████████████████| 1.5MB 29.3MB/s 
     |████████████████████████████████| 51kB 5.3MB/s 
     |████████████████████████████████| 20.1MB 1.4MB/s 
     |████████████████████████████████| 51kB 4.7MB/s 
     |████████████████████████████████| 8.2MB 16.7MB/s 
     |███████████████████████████████

In [54]:
!python -m deeppavlov install squad_bert
!python -m deeppavlov install ner_ontonotes

2021-07-07 19:38:41.602 INFO in 'deeppavlov.core.common.file'['file'] at line 32: Interpreting 'squad_bert' as '/usr/local/lib/python3.7/dist-packages/deeppavlov/configs/squad/squad_bert.json'
     |████████████████████████████████| 110.5MB 79kB/s 
     |████████████████████████████████| 3.8MB 25.7MB/s 
     |████████████████████████████████| 51kB 5.7MB/s 
     |████████████████████████████████| 512kB 31.8MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp37-none-any.whl size=7557 sha256=caafe5197ae37240814c1476dec84fe72c1252374fc919cf819c68753c34bc8a
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.12.1 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
ERROR: kapre 0.3.5 has requirement numpy>=1.18.5, but you'll have numpy 1.18.0 which is incompatible.
ERROR: kapre 0.3.5 has requirement tensorflow>=2.0.0, but you'll have tensorflow 1.15.2 wh

In [55]:
import deeppavlov
from deeppavlov import configs, build_model

In [56]:
deeppavlov_ner = build_model(configs.ner.ner_bert_ent_and_type_rus, download=True)
rus_document = "Нью-Йорк, США, 30 апреля 2020, 01:01 — REGNUM В администрации президента США Дональда Трампа планируют пройти все этапы создания вакцины от коронавируса в ускоренном темпе и выпустить 100 млн доз до конца 2020 года, передаёт агентство Bloomberg со ссылкой на осведомлённые источники"
deeppavlov_ner([rus_document])

2021-07-07 19:39:39.621 INFO in 'deeppavlov.core.data.utils'['utils'] at line 95: Downloading from http://files.deeppavlov.ai/kbqa/models/ner_cq_rus.tar.gz to /root/.deeppavlov/models/ner_cq_rus.tar.gz
100%|██████████| 1.32G/1.32G [02:33<00:00, 8.58MB/s]
2021-07-07 19:42:14.415 INFO in 'deeppavlov.core.data.utils'['utils'] at line 272: Extracting /root/.deeppavlov/models/ner_cq_rus.tar.gz archive into /root/.deeppavlov/models/ner_ent_and_type_rus
2021-07-07 19:42:30.588 INFO in 'deeppavlov.core.data.utils'['utils'] at line 95: Downloading from http://files.deeppavlov.ai/deeppavlov_data/bert/multi_cased_L-12_H-768_A-12.zip to /root/.deeppavlov/downloads/multi_cased_L-12_H-768_A-12.zip
100%|██████████| 663M/663M [01:08<00:00, 9.62MB/s]
2021-07-07 19:43:40.526 INFO in 'deeppavlov.core.data.utils'['utils'] at line 272: Extracting /root/.deeppavlov/downloads/multi_cased_L-12_H-768_A-12.zip archive into /root/.deeppavlov/downloads/bert_models
2021-07-07 19:43:49.709 INFO in 'deeppavlov.core.

2021-07-07 19:43:56.510 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /root/.deeppavlov/models/ner_ent_and_type_rus/tag.dict]











The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where




Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.


Instructions for updating:
Use standard file APIs to check for files with this prefix.


2021-07-07 19:44:29.118 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 51: [loading model from /root/.deeppavlov/models/ner_ent_and_type_rus/model]



INFO:tensorflow:Restoring parameters from /root/.deeppavlov/models/ner_ent_and_type_rus/model


[[['Нью',
   '-',
   'Йорк',
   ',',
   'США',
   ',',
   '30',
   'апреля',
   '2020',
   ',',
   '01',
   ':',
   '01',
   '—',
   'REGNUM',
   'В',
   'администрации',
   'президента',
   'США',
   'Дональда',
   'Трампа',
   'планируют',
   'пройти',
   'все',
   'этапы',
   'создания',
   'вакцины',
   'от',
   'коронавируса',
   'в',
   'ускоренном',
   'темпе',
   'и',
   'выпустить',
   '100',
   'млн',
   'доз',
   'до',
   'конца',
   '2020',
   'года',
   ',',
   'передаёт',
   'агентство',
   'Bloomberg',
   'со',
   'ссылкой',
   'на',
   'осведомлённые',
   'источники']],
 array([[[9.71761167e-01, 2.64181718e-02, 1.82059826e-03],
         [9.60046053e-01, 3.84359621e-02, 1.51804090e-03],
         [9.21502233e-01, 7.51483515e-02, 3.34933866e-03],
         [9.92245018e-01, 6.89126039e-03, 8.63717811e-04],
         [9.87261832e-01, 1.17831351e-02, 9.55077179e-04],
         [9.96919036e-01, 2.74484046e-03, 3.36058962e-04],
         [9.97517467e-01, 2.30196840e-03, 1.80645322e

#### 3.написать свой нер попробовать разные подходы (с доп информацией без) так же с учётом соседей и без них

In [57]:
!pip -q install spacy
!python -m spacy download en
!python -m spacy download en_core_web_sm

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.7/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.7/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [58]:
!pip install -U spacy
!python -m spacy info

     |████████████████████████████████| 6.4MB 4.2MB/s 
     |████████████████████████████████| 624kB 30.8MB/s 
     |████████████████████████████████| 460kB 35.6MB/s 
     |████████████████████████████████| 51kB 6.6MB/s 
     |████████████████████████████████| 10.1MB 39.0MB/s 
ERROR: deeppavlov 0.15.0 has requirement pydantic==1.3, but you'll have pydantic 1.8.2 which is incompatible.
  Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Found existing installation: pydantic 1.3
    Uninstalling pydantic-1.3:
      Successfully uninstalled pydantic-1.3
  Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-1.0.5
  Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
  Found existing installation: spacy 2.2.4
    Uninstalling spacy-2.2.4:
      Successfully uninstalled spacy-2.2.4

================

In [59]:
!python -m spacy download en_core_web_md

     |████████████████████████████████| 45.4MB 1.4MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


In [60]:
import spacy
from spacy import displacy
!pip install spacy model
!pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_md-2.2.0/en_core_web_md-2.2.0.tar.gz
import en_core_web_md

ERROR: Could not find a version that satisfies the requirement model (from versions: none)
ERROR: No matching distribution found for model
     |████████████████████████████████| 96.4MB 1.3MB/s 
  Created wheel for en-core-web-md: filename=en_core_web_md-2.2.0-cp37-none-any.whl size=98072936 sha256=e9655886ccae3f882144483e8994ab24ed0ad9ae0b2ba681b6c6ad0287b77b1c
  Stored in directory: /root/.cache/pip/wheels/5f/3e/c9/36dd6e13b449fd84cd1f94b72dfbc559daf09f53dbf4e697a3
Successfully built en-core-web-md
  Found existing installation: en-core-web-md 3.1.0
    Uninstalling en-core-web-md-3.1.0:
      Successfully uninstalled en-core-web-md-3.1.0


In [61]:
nlp = en_core_web_md.load()
ny_bb = url_to_string('https://www.nytimes.com/2018/08/13/us/politics/peter-strzok-fired-fbi.html?hp&action=click&pgtype=Homepage&clickSource=story-heading&module=first-column-region&region=top-news&WT.nav=top-news')
article = nlp(ny_bb)
displacy.render(article, jupyter=True, style='ent')

OSError: ignored